Код расчитан на то, что шейпы называются как в gdb

In [1]:
import arcpy
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np
from math import log10, floor, isfinite

Параметры

In [2]:
# ========= PATHS =========
region ='Yakutia'
gdb_path   = r"C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\Default.gdb"
RISKS_FDS  = "risk"       # hazards live here
ECON_FDS   = "economics"   # Economy lives here
ECON_FC    = f"{region}_economics_MO" 

# ========= MAPPING & CONFIG =========
hazard_mapping = {
    "economics": "Economy",
    "hazCold": "Cold",
    "hazDrought": "Drought",
    "hazWildfire": "Wildfire",
    "hazFlood": "Flood",
    "hazFreez5": "Freeze",
    "hazHail": "Hail",
    "hazHeat": "Heat",
    "hazRain": "Precipitation",     # (spelling fixed)
    "hazThunderstorm": "Thunder",
    "hazWind": "Wind",
    "hazPermafrost": "Permafrost",
}

category_order = ["неопасно","умерено опасно","опасно","весьма опасно","очень опасно"]
asset_types = ["ho", "ro", "fo", "ag", "pop"]

# ========= HELPERS =========
def extract_short_code(fc_name: str) -> str:
    """
    Try to pull a 'hazXXXX' token from the FC name.
    Example: 'Yakutia_hazCold_ExportFeature' -> 'hazCold'
    Fallback: if a known key is a substring of the name, use that.
    Else: return the whole name.
    """
    m = re.search(r"(haz[A-Za-z0-9]+)", fc_name)
    if m:
        return m.group(1)
    for key in hazard_mapping.keys():
        if key.lower() in fc_name.lower():
            return key
    return fc_name

# ========= COLLECT HAZARDS FROM GDB =========
hazard_data = {}  # dict[str, str] => { "Wildfire": "<full path to FC>", ... }

# 1) Hazards inside RISKS_FDS
arcpy.env.workspace = os.path.join(gdb_path, RISKS_FDS)
print(arcpy.env.workspace)
print(arcpy.ListFeatureClasses())
for fc in arcpy.ListFeatureClasses() or []:
    name_l = fc.lower()
    if "template" in name_l or "perm_hazardtemplate_mo" in name_l:
        print('template')
        continue  # skip your template FC

    short = extract_short_code(fc)
    hazard_name = hazard_mapping.get(short, short)
    full_path = os.path.join(gdb_path, RISKS_FDS, fc)
    print(full_path)
    hazard_data[hazard_name] = full_path

# 2) Economy from ECON_FDS (explicit FC name you provided)
econ_path = os.path.join(gdb_path, ECON_FDS, ECON_FC)
print(econ_path)
if arcpy.Exists(econ_path):
    hazard_data["Economy"] = econ_path
else:
    print(f"⚠️ Economy feature class not found at: {econ_path}")

print("Hazards loaded:", list(hazard_data.keys()))

# ========= TOTALS FROM ECONOMY =========
total_prices = {}
if "Economy" in hazard_data:
    # use your actual field names from the field listing
    econ_fields = {
        "ho": "hoPriceTot",
        "ro": "roPriceTot",
        "fo": "foPriceTotal",  # note: 'Total'
        "ag": "agPriceTotal",  # note: 'Total'
        "pop": "popTotal",
    }

    # verify fields exist; raise clear error otherwise
    econ_fields_exist = {f.name for f in arcpy.ListFields(hazard_data["Economy"])}
    missing = [v for v in econ_fields.values() if v not in econ_fields_exist]
    if missing:
        raise RuntimeError(f"Economy is missing fields: {missing}")

    targets = [econ_fields[k] for k in ("ho","ro","fo","ag","pop")]
    sums = {k: 0.0 for k in ("ho","ro","fo","ag","pop")}

    with arcpy.da.SearchCursor(hazard_data["Economy"], targets) as cur:
        for row in cur:
            for k, val in zip(("ho","ro","fo","ag","pop"), row):
                if val is None:
                    continue
                try:
                    sums[k] += float(val)
                except (TypeError, ValueError):
                    try:
                        sums[k] += float(str(val).replace(" ", "").replace(",", "."))
                    except:
                        pass

    total_prices = sums
    print("\nTotal asset values (from Economy):")
    for k in ("ho","ro","fo","ag","pop"):
        print(f"  {k}: {total_prices[k]:,.0f}")
else:
    print("\n⚠️ Economy not loaded; totals skipped.")

econ_fc  = hazard_data["Economy"]    
def fc_to_df(fc_path: str) -> pd.DataFrame:
    """Read non-geometry fields into a DataFrame (NULL-safe)."""
    fld_objs = [f for f in arcpy.ListFields(fc_path) if f.type not in ("Geometry","Raster")]
    names = [f.name for f in fld_objs]
    if not names:
        return pd.DataFrame()
    nulls = {}
    for f in fld_objs:
        t = f.type
        nulls[f.name] = (-1 if t in ("SmallInteger","Integer","OID")
                         else (np.nan if t in ("Single","Double")
                               else ("" if t in ("String","Guid") else None)))
    try:
        arr = arcpy.da.TableToNumPyArray(econ_fc, names, skip_nulls=False, null_value=nulls)
        eco = pd.DataFrame(arr)
    except Exception:
        rows = []
        with arcpy.da.SearchCursor(econ_fc, names) as cur:
            for rec in cur:
                rows.append({n: (v if v is not None else nulls.get(n)) for n, v in zip(names, rec)})
        eco = pd.DataFrame(rows)
    # normalize strings
    for c in eco.columns:
        if eco[c].dtype.kind in ("U","S","O"):
            eco[c] = eco[c].astype(object)
    return eco

def norm_key(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

CAND_KEYS = ("MO", "MO_NAME")
def pick_join_key(hgdf: pd.DataFrame, eco_df: pd.DataFrame) -> str:
    for k in CAND_KEYS:
        if k in hgdf.columns and k in eco_df.columns:
            return k
    common = [c for c in hgdf.columns if c in eco_df.columns and c.lower() != "geometry"]
    if common:
        return common[0]
    raise ValueError("No common municipality key between hazard CSV and Economy.")

# Build the Economy DataFrame used by the donuts code
eco = fc_to_df(econ_fc)

# Economy field names (match your GDB schema)
price_cols = {
    "ho": "hoPriceTot",
    "ro": "roPriceTot",
    "fo": "foPriceTotal",   # note 'Total'
    "ag": "agPriceTotal",   # note 'Total'
    "pop": "popTotal",
}


C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\Default.gdb\risk
['Yakutia_hazardTemplate_MO', 'Yakutia_hazHeat_MO', 'Yakutia_hazFreez5_MO', 'Yakutia_hazDrought_MO', 'Yakutia_hazThunderstorm_MO', 'Yakutia_hazHail_MO', 'Yakutia_hazRain_MO', 'Yakutia_hazWind_MO', 'Yakutia_hazCold_MO', 'Yakutia_hazFlood_MO', 'Yakutia_hazPermafrost_MO', 'Yakutia_hazWildfire_MO']
template
C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\Default.gdb\risk\Yakutia_hazHeat_MO
C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\Default.gdb\risk\Yakutia_hazFreez5_MO
C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\Default.gdb\risk\Yakutia_hazDrought_MO
C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\Default.gdb\risk\Yakutia_hazTh

Функции

In [3]:
category_order = [
    "неопасно",
    "умерено опасно",
    "опасно",
    "весьма опасно",
    "очень опасно"
]
asset_types = ["ho", "ro", "fo", "ag", "pop"]

def summarize_by_hazard(hazard_name: str, fc_path: str, total_prices: dict) -> dict:
    """
    fc_path = full path to a feature class in the GDB (e.g., D:\...\Default.gdb\risks\Yakutia_hazWildfire_...).
    total_prices = dict with totals for ho/ro/fo/ag/pop from Economy.
    Returns a single summary row (dict) for this hazard.
    """
    row = {"hazard": hazard_name}

    # --- 0) discover available fields once ---
    fields_set = {f.name for f in arcpy.ListFields(fc_path)}

    # --- 1) averages for % fields, sums for absolute fields ---
    percent_cols = ["rNF_ho", "rSF_ho", "rLF_ro", "rSF_ag", "rSF_fo", "rNF_pop"]
    abs_cols     = ["rNiNA_ho","rSiHA_ho","rLiHA_ro","rSiHA_ag","rSiHA_fo","rNiHA_pop", 'rNaR_pop']

    # Build a minimal field list for the cursor
    needed_for_stats = [c for c in percent_cols + abs_cols if c in fields_set]

    # --- 2) category column & damage columns by asset ---
    cat_col = "hNormTot" if hazard_name == "Flood" else "hExpertTot"
    has_cat = cat_col in fields_set

    dmg_cols_map = {}
    for a in asset_types:
        if hazard_name == "Wildfire":
            dmg_cols_map[a] = f"rNaR_{a}" if a == "pop" else f"rVaR_{a}"
        else:
            dmg_cols_map[a] = f"rNiHA_{a}" if a == "pop" else f"rViHA_{a}"

    # Which of those actually exist?
    dmg_cols_present = {a: c for a, c in dmg_cols_map.items() if c in fields_set}

    # Final cursor field list: stats + (cat_col if present) + all present dmg cols
    cursor_fields = []
    cursor_fields.extend(needed_for_stats)
    if has_cat:
        cursor_fields.append(cat_col)
    cursor_fields.extend(sorted(set(dmg_cols_present.values())))  # unique

    if not cursor_fields:
        # still emit shape for table: zeros and pct/value columns
        for c in percent_cols:
            row[f"{c}_pct"] = 0.0
        for c in abs_cols:
            row[c] = 0.0
        for a in asset_types:
            dmg_col = dmg_cols_map[a]
            row[dmg_col] = 0.0
            for i, _cat in enumerate(category_order):
                row[f"{dmg_col}_pct_{i}"] = 0.0
                row[f"{dmg_col}_val_{i}"] = 0.0
        return row


    # --- accumulators ---
    pct_sums = {c: 0.0 for c in percent_cols if c in fields_set}
    pct_counts = {c: 0 for c in percent_cols if c in fields_set}
    abs_sums = {c: 0.0 for c in abs_cols if c in fields_set}

    # per-asset, per-category sums
    by_asset_by_cat = {
        a: {cat: 0.0 for cat in category_order} for a in asset_types
    }

    # index mapping for fast access
    idx = {name: i for i, name in enumerate(cursor_fields)}
    # Iterate rows once
    with arcpy.da.SearchCursor(fc_path, cursor_fields) as cur:
        for rec in cur:
            # 1) percent means
            for c in pct_sums.keys():
                v = rec[idx[c]]
                if v is not None:
                    try:
                        fv = float(v)
                        if isfinite(fv):
                            pct_sums[c] += fv
                            pct_counts[c] += 1
                    except Exception:
                        pass

            # 2) abs sums
            for c in abs_sums.keys():
                v = rec[idx[c]]
                if v is not None:
                    try:
                        abs_sums[c] += float(v)
                    except Exception:
                        # try forgiving parse
                        try:
                            abs_sums[c] += float(str(v).replace(" ", "").replace(",", "."))
                        except Exception:
                            pass

            # 3) category splits (only if we have a category col)
            if has_cat:
                cat_val = rec[idx[cat_col]]
                # keep as-is; if None or not one of the known categories, treat as "неопасно"
                cat_key = cat_val if cat_val in category_order else "неопасно"

                for a, dmg_col in dmg_cols_present.items():
                    v = rec[idx[dmg_col]]
                    if v is None:
                        continue
                    try:
                        add = float(v)
                    except Exception:
                        try:
                            add = float(str(v).replace(" ", "").replace(",", "."))
                        except Exception:
                            add = 0.0
                    by_asset_by_cat[a][cat_key] += add

    # write percent means (%)
    for c in percent_cols:
        if c in pct_sums:
            n = pct_counts.get(c, 0)
            mean_val = (pct_sums[c] / n) if n > 0 else 0.0
            row[f"{c}_pct"] = float(mean_val * 100.0)  # your original *100
        else:
            row[f"{c}_pct"] = 0.0

    # write abs sums
    for c in abs_cols:
        row[c] = float(abs_sums.get(c, 0.0))

    # 4) per-asset totals + category shares + absolute values
    # 4) per-asset totals + category shares + absolute values
    danger_cats = category_order[1:]  # ["умерено опасно","опасно","весьма опасно","очень опасно"]

    for a in asset_types:
        dmg_col = dmg_cols_map[a]

        # absolute affected amounts per hazard category (aggregated over rows)
        sums_by_cat = by_asset_by_cat.get(a, {cat: 0.0 for cat in category_order})

        # total affected (across ALL cats as read from the FC; for "неопасно" it should be 0 anyway)
        affected_total = float(sum(sums_by_cat.get(cat, 0.0) for cat in danger_cats))

        # keep the total affected for this asset (same meaning as before)
        row[dmg_col] = affected_total

        # Economy denominator for the whole region
        tot = float(total_prices.get(a, 0.0))

        # --- values per category ---
        # cats 1..4: already have affected values
        # cat 0 ("неопасно"): residual = total - affected_total (clip at 0)
        safe_val = max(tot - affected_total, 0.0) if tot > 0 else 0.0

        # write absolute values
        for i, cat in enumerate(category_order):
            if i == 0:
                row[f"{dmg_col}_val_{i}"] = safe_val
            else:
                row[f"{dmg_col}_val_{i}"] = float(sums_by_cat.get(cat, 0.0))

        # --- percent shares per category ---
        if tot > 0:
            # cats 1..4 from affected / tot
            shares = {cat: (sums_by_cat.get(cat, 0.0) / tot) * 100.0 for cat in danger_cats}
            # cat 0 from residual to close to 100%
            shares_others_sum = sum(shares.values())
            shares_neopasno = max(100.0 - shares_others_sum, 0.0)
        else:
            shares = {cat: 0.0 for cat in danger_cats}
            shares_neopasno = 0.0

        # write percents in fixed 0..4 order
        for i, cat in enumerate(category_order):
            if i == 0:
                row[f"{dmg_col}_pct_{i}"] = float(shares_neopasno)
            else:
                row[f"{dmg_col}_pct_{i}"] = float(shares.get(cat, 0.0))


    return row

# ---------- formatting helpers (unchanged from your spec) ----------
def _round_sig(x: float, sig: int = 3) -> float:
    if not isfinite(x) or x == 0:
        return 0.0
    return round(x, sig - 1 - floor(log10(abs(x))))

def _fmt_clean(x: float) -> str:
    s = f"{x:.10f}".rstrip('0').rstrip('.')
    return s if s else "0"

def format_center(asset: str, value: float) -> str:
    if not np.isfinite(value):
        return ""
    if value == 0:
        return "0"

    unit = "чел." if asset == "pop" else "₽"
    abs_v = abs(value)
    if abs_v >= 1_000_000_000_000:
        scaled = value / 1_000_000_000_000.0
        return f"{_fmt_clean(_round_sig(scaled, 3))} трлн. {unit}"
    elif abs_v >= 1_000_000_000:
        scaled = value / 1_000_000_000.0
        return f"{_fmt_clean(_round_sig(scaled, 3))} млрд. {unit}"
    elif abs_v >= 1_000_000:
        scaled = value / 1_000_000.0
        return f"{_fmt_clean(_round_sig(scaled, 3))} млн. {unit}"
    else:
        scaled = value / 1_000.0
        return f"{_fmt_clean(_round_sig(scaled, 3))} тыс. {unit}"


In [4]:
# === Build the table ===
rows = []
for hz, fc_path in hazard_data.items():
    if hz == "Economy":
        continue
    print(f"Summarizing {hz} …")
    
    row = summarize_by_hazard(hz, fc_path, total_prices)
    row["region"] = region
    rows.append(row)


summary = pd.DataFrame(rows)

# round all percentage columns to 1 decimal
for col in summary.columns:
    if "_pct" in col:
        summary[col] = summary[col].round(1)

# === Save CSV next to the GDB (not inside it) ===
out_dir = os.path.dirname(gdb_path)  # one level above the .gdb
out_csv = os.path.join(out_dir, "hazard_summary_by_risk.csv")

# Save with UTF-8 encoding (handles Cyrillic safely)
summary.to_csv(out_csv, index=False, encoding="utf-8-sig")

print(f"✅ Saved summary outside GDB:\n{out_csv}")


Summarizing Heat …
Summarizing Freeze …
Summarizing Drought …
Summarizing Thunder …
Summarizing Hail …
Summarizing Precipitation …
Summarizing Wind …
Summarizing Cold …
Summarizing Flood …
Summarizing Permafrost …
Summarizing Wildfire …
✅ Saved summary outside GDB:
C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\hazard_summary_by_risk.csv


Далее код, создающий колонки nodata = 100 для отсутствующих значений процентов

In [5]:
out_dir = Path(gdb_path).parent / "out_with_pct_Yakutia_nodata"
out_dir.mkdir(parents=True, exist_ok=True)

# --- config ---
category_order = ["неопасно","умерено опасно","опасно","весьма опасно","очень опасно"]
cat_to_idx = {name: i for i, name in enumerate(category_order)}
asset_types = ["ho","ro","fo","ag","pop"]
econ_fc = hazard_data["Economy"]

price_cols = {
    "ho": "hoPriceTot",
    "ro": "roPriceTot",
    "fo": "foPriceTotal",   # note: Total
    "ag": "agPriceTotal",   # note: Total
    "pop": "popTotal",
}
percent_cols = ["rNF_ho","rSF_ho","rLF_ro","rSF_ag","rSF_fo","rNF_pop"]
CAND_KEYS = ["MO_NAME","MO"]

def asset_cols(asset: str):
    return [f"{asset}_pct_{i}" for i in range(5)] + [f"{asset}_pct_nodata"]

def asset_val_cols(asset: str):
    return [f"{asset}_val_{i}" for i in range(5)]

def pick_join_key(hdf: pd.DataFrame, econ_fc: pd.DataFrame) -> str:
    for k in CAND_KEYS:
        if k in hdf.columns and k in econ_fc.columns:
            return k
    common = [c for c in hdf.columns if c in econ_fc.columns and c.lower() != "geometry"]
    if common:
        return common[0]
    raise ValueError("No common municipality key found between hazard and Economy.")

def norm_key(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

def fc_to_df(fc_path: str) -> pd.DataFrame:
    """
    Read all non-geometry fields; handle NULLs so TableToNumPyArray doesn't fail on ints.
    Falls back to SearchCursor if needed.
    """
    fld_objs = [f for f in arcpy.ListFields(fc_path) if f.type not in ("Geometry","Raster")]
    fields = [f.name for f in fld_objs]
    if not fields:
        return pd.DataFrame()

    null_value = {}
    for f in fld_objs:
        t = f.type
        if t in ("SmallInteger","Integer","OID"):
            null_value[f.name] = -1
        elif t in ("Single","Double"):
            null_value[f.name] = np.nan
        elif t in ("String","Guid"):
            null_value[f.name] = ""
        elif t == "Date":
            null_value[f.name] = None
        else:
            null_value[f.name] = ""

    try:
        arr = arcpy.da.TableToNumPyArray(
            in_table=fc_path,
            field_names=fields,
            skip_nulls=False,
            null_value=null_value
        )
        df = pd.DataFrame(arr)
    except Exception:
        # Fallback: SearchCursor row-by-row with the same null defaults
        rows = []
        with arcpy.da.SearchCursor(fc_path, fields) as cur:
            for rec in cur:
                d = {}
                for name, val in zip(fields, rec):
                    if val is None:
                        d[name] = null_value.get(name, None)
                    else:
                        d[name] = val
                rows.append(d)
        df = pd.DataFrame(rows)

    # Normalize string dtypes
    for c in df.columns:
        if df[c].dtype.kind in ("U","S","O"):
            df[c] = df[c].astype(object)
    return df

# --- Economy DF (for joins + region + denominators) ---
eco_fc = hazard_data["Economy"]
eco_df = fc_to_df(eco_fc)
region_name = (
    eco_df["region"].dropna().astype(str).unique()[0]
    if "region" in eco_df.columns and eco_df["region"].notna().any()
    else "Unknown"
)

# ============ PER-HAZARD MUNICIPALITY TABLES (no combined file) ============
for hazard_name, fc_path in hazard_data.items():
    if hazard_name == "Economy":
        continue

    gdf = fc_to_df(fc_path)

    # Scale % columns idempotently
    for c in percent_cols:
        if c in gdf.columns:
            s = pd.to_numeric(gdf[c], errors="coerce")
            finite = s[np.isfinite(s)]
            if not finite.empty and (finite <= 1.0).mean() >= 0.90:
                gdf[c] = (s * 100.0).round(1)
            else:
                gdf[c] = s.round(1)

    key = pick_join_key(gdf, eco_df)

    gdf = gdf.copy()
    eco_tmp = eco_df[[k for k in [key,"region"] if k in eco_df.columns] + list(price_cols.values())].copy()

    gdf["_join_key"] = norm_key(gdf[key])
    eco_tmp["_join_key"] = norm_key(eco_tmp[key])
    eco_idx = eco_tmp.set_index("_join_key")

    # Municipality totals per asset
    mo_totals = {
        a: (eco_idx[price_cols[a]] if price_cols[a] in eco_idx.columns else pd.Series(dtype=float))
        for a in asset_types
    }

    # Region
    if "region" in eco_idx.columns:
        gdf["region"] = gdf["_join_key"].map(eco_idx["region"]).fillna(region_name)
    else:
        gdf["region"] = region_name

    # Category column
    cat_raw = gdf.get("hNormLive") if hazard_name == "Flood" else gdf.get("hExpertTot")
    cat_idx = pd.Series(0, index=gdf.index, dtype=int)
    if cat_raw is not None:
        cat_to_idx_lower = {k.lower(): v for k, v in cat_to_idx.items()}
        cat_idx = cat_raw.astype(str).str.lower().map(cat_to_idx_lower).fillna(0).astype(int)
    gdf["_cat_idx"] = cat_idx.clip(0, 4)

    # Initialize share (pct) and value columns
    for a in asset_types:
        for col in asset_cols(a):
            gdf[col] = 0.0
        for col in asset_val_cols(a):
            gdf[col] = 0.0

    for a in asset_types:
        if hazard_name == "Wildfire":
            dmg_col = f"rNaR_{a}" if a == "pop" else f"rVaR_{a}"
        else:
            dmg_col = f"rNiHA_{a}" if a == "pop" else f"rViHA_{a}"

        denom = gdf["_join_key"].map(mo_totals[a])
        has_assets = denom.fillna(0) > 0

        exists_col = f"{a}_exists"
        gdf[exists_col] = has_assets.astype(bool)

        tok = re.compile(rf"(^|_){re.escape(a)}(_|$)")
        candidate_cols = [c for c in gdf.columns if tok.search(c)]
        wipe_cols = [c for c in candidate_cols if not c.endswith("_exists")]

        mask_no_assets = ~has_assets
        for col in wipe_cols:
            if pd.api.types.is_bool_dtype(gdf[col].dtype):
                gdf.loc[mask_no_assets, col] = False
            else:
                gdf.loc[mask_no_assets, col] = np.nan

        # absolute affected value for this asset in this MO
        if dmg_col in gdf.columns:
            num = pd.to_numeric(gdf[dmg_col], errors="coerce")
        else:
            num = pd.Series(np.nan, index=gdf.index)

        # % affected of municipal total
        pct = pd.Series(np.nan, index=gdf.index)
        valid_mask = has_assets & (~pd.isna(num)) & (denom > 0)
        pct.loc[valid_mask] = (num.loc[valid_mask] / denom.loc[valid_mask]) * 100.0
        pct = pct.clip(lower=0.0, upper=100.0).round(1)

        # % and value of the SAFE part
        pct_safe = (100.0 - pct).clip(lower=0.0, upper=100.0).round(1)
        val_safe = (denom - num).where(valid_mask, np.nan)
        val_safe = val_safe.mask(val_safe < 0, 0.0)  # clip negatives to 0

        pct_cols = [f"{a}_pct_{i}" for i in range(5)]  # без nodata
        val_cols = [f"{a}_val_{i}" for i in range(5)]

        # write category-specific cells
        for i in range(5):
            mask_cat = (gdf["_cat_idx"] == i) & has_assets
            if i == 0:
                # "неопасно" column gets the SAFE share/value
                gdf.loc[mask_cat, pct_cols[0]] = pct_safe.loc[mask_cat]
                gdf.loc[mask_cat, val_cols[0]] = val_safe.loc[mask_cat]
            else:
                # dangerous category gets affected share/value
                gdf.loc[mask_cat, pct_cols[i]] = pct.loc[mask_cat]
                gdf.loc[mask_cat, val_cols[i]] = num.loc[mask_cat]

        # No extra residual math needed now; we've set pct_0 directly.

        # residual 
        residual = (100.0 - pct).clip(lower=0.0, upper=100.0)

        # --- absolute residual goes to val_0 ---
        # denom = total municipal asset value; num = affected/at-risk absolute value
        residual_abs = (denom - num)
        residual_abs = residual_abs.where(residual_abs > 0, 0.0)  # clip at 0

        mask_cat0  = (gdf["_cat_idx"] == 0) & has_assets
        mask_non0  = (gdf["_cat_idx"] != 0) & has_assets

        # If municipality is "неопасно": all value is non-dangerous → val_0 = total
        gdf.loc[mask_cat0, val_cols[0]] = denom.loc[mask_cat0]

        # If municipality is in category 1..4: split into dangerous vs safe
        #   - val_{cat} = num (already set above)
        #   - val_0     = denom - num (unaffected)
        gdf.loc[mask_non0, val_cols[0]] = residual_abs.loc[mask_non0]

        # keep your existing % residual behavior (affects only pct_0)
        mask_non0_pct = (gdf["_cat_idx"] != 0) & (residual > 0) & has_assets
        gdf.loc[mask_non0_pct, pct_cols[0]] += residual.loc[mask_non0_pct]

        # ===== ИСПРАВЛЕННЫЙ КОД: Правильная логика для nodata =====
        # Список всех pct колонок (включая pct_0!)
        all_pct_cols = [f"{a}_pct_{i}" for i in range(5)]
        
        # Заменяем только NULL/NaN на 0 в pct колонках, но не перезаписываем существующие значения
        for col in all_pct_cols:
            if col in gdf.columns:
                # Заменяем только NaN/None на 0, сохраняя существующие значения
                gdf[col] = gdf[col].fillna(0.0)
        
        # Заменяем NULL в val колонках
        for col in val_cols:
            if col in gdf.columns:
                gdf[col] = gdf[col].fillna(0.0)
        
        # Проверяем что ВСЕ pct колонки (0-4) равны 0
        all_pct_zero_mask = True
        for col in all_pct_cols:
            if col in gdf.columns:
                all_pct_zero_mask = all_pct_zero_mask & (gdf[col] == 0)
        
        # Устанавливаем значения для *_pct_nodata
        nodata_col = f"{a}_pct_nodata"
        
        # Для строк где есть активы
        if has_assets.any():
            # Если ВСЕ pct колонки (0-4) = 0, то nodata = 100, иначе 0
            gdf.loc[has_assets & all_pct_zero_mask, nodata_col] = 100.0
            gdf.loc[has_assets & (~all_pct_zero_mask), nodata_col] = 0.0
        else:
            # Если активов нет, устанавливаем nodata в 0
            gdf.loc[:, nodata_col] = 0.0

                    # ===== ДОПОЛНИТЕЛЬНО: Гарантируем что в nodata нет NULL =====
        # Заменяем все оставшиеся NULL в nodata колонке
        gdf[nodata_col] = gdf[nodata_col].fillna(100)

    # Cleanup
    drop_cols = [
        "_join_key","_cat_idx","Shape_Length","Shape_Area","Shape_Leng","geometry",
        "area","areaLive","areaLiveF","sTot","sTotFrac","sLive","sLiveFrac","sDensity",
        "spTot","spTotFrac","spLive","spLiveFrac","ieMeanTot","ieMaxTot","ieMeanLiv","ieMaxLiv",
        "itMeanTot","itMeanLive","iOverTot","iOverLiv","fdMeanTot","fdMaxTot","fdMeanLive","fdMaxLive",
        "feMeanTot","feMeanLive","hFxSTot","hFxSLive","Риски","Риск_1",
    ]
    gdf = gdf.drop(columns=drop_cols, errors="ignore")
    gdf["hazard"] = hazard_name

    # Cast numeric; round pct columns
    for c in gdf.columns:
        if pd.api.types.is_bool_dtype(gdf[c].dtype):
            continue
        if pd.api.types.is_integer_dtype(gdf[c]) or pd.api.types.is_float_dtype(gdf[c]):
            gdf[c] = pd.to_numeric(gdf[c], errors="coerce").astype("float64")
    pct_like = [c for c in gdf.columns if "_pct" in c]
    if pct_like:
        gdf[pct_like] = gdf[pct_like].round(1)

    # Save per-hazard CSV (outside gdb)
    out_path = out_dir / f"{hazard_name}_withPct.csv"
    gdf.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"✅ Saved: {hazard_name} -> {out_path}")

✅ Saved: Heat -> C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\out_with_pct_Yakutia_nodata\Heat_withPct.csv
✅ Saved: Freeze -> C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\out_with_pct_Yakutia_nodata\Freeze_withPct.csv
✅ Saved: Drought -> C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\out_with_pct_Yakutia_nodata\Drought_withPct.csv
✅ Saved: Thunder -> C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\out_with_pct_Yakutia_nodata\Thunder_withPct.csv
✅ Saved: Hail -> C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\out_with_pct_Yakutia_nodata\Hail_withPct.csv
✅ Saved: Precipitation -> C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\out_wi

In [7]:
import os
import matplotlib
# Force a headless backend so it never crashes on GUI issues
matplotlib.use("Agg")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

# Cyrillic-safe font (for labels like "неопасно")
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["svg.fonttype"] = "none"


In [17]:

base_dir = Path(gdb_path).parent
summary = pd.read_csv(base_dir / "hazard_summary_by_risk.csv", encoding="utf-8-sig")
# === Settings ===
asset_types = ["ho","ro","fo","ag","pop"]
category_order = ["неопасно", "умерено опасно", "опасно", "весьма опасно", "очень опасно"]
category_colors = {
    "неопасно": "#a5de94",
    "умерено опасно": "#feea80",
    "опасно": "#fec979",
    "весьма опасно": "#fb8066",
    "очень опасно": "#f65356",
}
plt.rcParams['svg.fonttype'] = 'none'

def _base_prefix(hazard: str, asset: str) -> str:
    """Return rViHA / rNiHA / rVaR depending on hazard & asset."""
    if str(hazard) == "Wildfire":
        if asset != "pop":
            return "rVaR"
        else: return 'rNaR'
    return "rNiHA" if asset == "pop" else "rViHA"

def share_cols(hazard: str, asset: str) -> list[str]:
    """rXX_{asset}_pct_0..4 in the order of category_order."""
    base = _base_prefix(hazard, asset)
    return [f"{base}_{asset}_pct_{i}" for i in range(len(category_order))]

def total_key(hazard: str, asset: str) -> str:
    """rXX_{asset} total."""
    base = _base_prefix(hazard, asset)
    return f"{base}_{asset}"

def autopct_fmt(p: float) -> str:
    if p <= 0:
        return ''
    elif p >= 1:
        return f"{p:.0f}%"
    elif p >= 0.1:
        return f"{p:.1f}%"
    else:
        return f"{p:.2f}%"

num_hazards = len(summary)
fig, axes = plt.subplots(num_hazards, len(asset_types), figsize=(22, 5 * max(1, num_hazards)))

# normalize axes to 2D
if num_hazards == 1 and len(asset_types) == 1:
    axes = np.array([[axes]])
elif num_hazards == 1:
    axes = np.array(axes)[np.newaxis, :]
elif len(asset_types) == 1:
    axes = np.array(axes)[:, np.newaxis]

for row_idx, (_, hz_row) in enumerate(summary.iterrows()):
    hazard_name = hz_row.get("hazard", f"Hazard {row_idx+1}")

    for col_idx, a in enumerate(asset_types):
        ax = axes[row_idx, col_idx]

        # shares
        cols = share_cols(hazard_name, a)
        shares = [float(hz_row.get(c, 0) or 0) for c in cols]
        colors = [category_colors[c] for c in category_order]

        if sum(shares) <= 0:
            ax.pie([1.0], labels=None, colors=["#eeeeee"], startangle=90,
                   counterclock=True, wedgeprops=dict(width=0.5))
        else:
            ax.pie(
            shares,
            labels=None,
            autopct=autopct_fmt,
            colors=colors,
            startangle=90,
            counterclock=True,
            wedgeprops=dict(width=0.5),
            pctdistance=1.2,
            textprops=dict(fontsize=16),
        )

        # center label = global total (from Economy)
        ax.text(0, 0, format_center(a, total_prices[a]),
                ha='center', va='center',
                fontsize=16, weight='bold')

        ax.set_title(f"{hz_row.get('hazard', f'Hazard {row_idx+1}')} — {a}", fontsize=28)

# Legend & save
# fig.legend(category_order, loc='center left', bbox_to_anchor=(0.98, 0.5), title="Категории")
plt.tight_layout(rect=[0, 0, 0.95, 1])
out_dir = Path(f"{base_dir}/out_with_pct"); out_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(out_dir / "hazard_donuts_all_with_pop.svg", bbox_inches='tight')
plt.savefig(out_dir / "hazard_donuts_all_with_pop.png", bbox_inches='tight', dpi=200)
plt.show()

<ipython-input-17-1425890ad5b7>:94: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [19]:
econ_fc = hazard_data["Economy"]

# ---------- paths & input ----------
subfolder = f'{base_dir}/out_with_pct/'
for hazard_name in hazard_data.keys():
    print(hazard_name)
    if hazard_name =='Economy':
        continue
    else: risk = hazard_name
    in_path = f"{subfolder}{risk}_withPct.csv"

    # ---------- load (ignore geometry to avoid multipart errors) ----------
    gdf = pd.read_csv(in_path)

    # ---------- config ----------
    # Category order (0..4) maps to danger levels below
    category_order = ["неопасно", "умерено опасно", "опасно", "весьма опасно", "очень опасно"]
    category_colors = {
        "неопасно": "#a5de94",
        "умерено опасно": "#feea80",
        "опасно": "#fec979",
        "весьма опасно": "#fb8066",
        "очень опасно": "#f65356"
    }
    cat_idx_to_name = {i: name for i, name in enumerate(category_order)}

    # Assets to try to render (will be filtered by availability)
    all_assets = ["ho", "ro", "fo", "ag", "pop"]

    # Absolute totals for the donut center
    center_value_cols = {
        "ho":  "rViHA_ho",
        "ro":  "rViHA_ro",
        "fo":  "rViHA_fo",
        "ag":  "rViHA_ag",
        "pop": "rNiHA_pop",  # people
    }

    if hazard_name =='Wildfire':
        center_value_cols = {
        "ho":  "rVaR_ho",
        "ro":  "rVaR_ro",
        "fo":  "rVaR_fo",
        "ag":  "rVaR_ag",
        "pop": "rNaR_pop",  # people
    }

    # Detect municipality column
    mo_col = "MO" if "MO" in gdf.columns else None
    if mo_col is None:
        for cand in ("MO_NAME", "municipality", "МО"):
            if cand in gdf.columns:
                mo_col = cand
                break
    assert mo_col is not None, "Не найден столбец с муниципалитетом (например, 'MO')."

    # ✅ Use the Economy DataFrame here (NOT the FC path string)
    key = pick_join_key(gdf, eco)                 # was: pick_join_key(gdf, econ_fc)
    eco_tmp = eco[[key] + list(price_cols.values())].copy()  # was: econ_fc[[...]]
    eco_tmp["_join_key"] = norm_key(eco_tmp[key])
    eco_idx = eco_tmp.set_index("_join_key")

    # one lookup Series per asset, indexed by normalized join key
    mo_price_lookup = {
        a: (eco_idx[price_cols[a]] if price_cols[a] in eco_idx.columns else pd.Series(dtype=float))
        for a in ["ho","ro","fo","ag","pop"]
    }

    # population too, if you want to center-show pop totals sometimes
    mo_price_lookup["pop"] = eco_idx.get("popTotal", pd.Series(dtype=float))

    # ---------- figure out which assets are fully available ----------
    present_assets = []
    pct_cols_by_asset = {}
    for a in all_assets:
        pct_cols = [f"{a}_pct_{i}" for i in range(5)]
        if all(col in gdf.columns for col in pct_cols) and (center_value_cols[a] in gdf.columns):
            present_assets.append(a)
            pct_cols_by_asset[a] = pct_cols

    if not present_assets:
        raise RuntimeError("Нет ни одного актива с полным набором *_pct_0..4 и абсолютным столбцом для центра.")

    # ---------- plotting ----------
    mos = sorted(gdf[mo_col].astype(str).unique().tolist())
    n_rows, n_cols = len(mos), len(present_assets)

    plt.rcParams['svg.fonttype'] = 'none'
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(22, max(3.0, 2.2 * n_rows)))

    # normalize axes to 2D indexable array
    if n_rows == 1 and n_cols == 1:
        axes = np.array([[axes]])
    elif n_rows == 1:
        axes = np.array([axes])
    elif n_cols == 1:
        axes = np.array([[ax] for ax in axes])

    # set column titles (assets) only on the top row
    for c, a in enumerate(present_assets):
        axes[0][c].set_title(a, fontsize=13)

    for r, mo in enumerate(mos):
        df_mo = gdf[gdf[mo_col].astype(str) == mo]

        for c, a in enumerate(present_assets):
            ax = axes[r][c]

            # municipality key for Economy lookup
            mo_key_norm = norm_key(pd.Series([df_mo[mo_col].iloc[0]])).iloc[0]
            center_val = mo_price_lookup[a].get(mo_key_norm, np.nan)

            # --- CASE A: no assets in this muni → gray ring, no %
            no_assets = (not np.isfinite(center_val)) or (float(center_val) <= 0)
            if no_assets:
                ax.pie([1.0], labels=None, colors=["#e0e0e0"], startangle=90,
                    counterclock=True, wedgeprops=dict(width=0.5))
                ax.text(0, 0, "", ha="center", va="center", fontsize=11, weight="bold")
                continue  # skip share logic

            # --- Build shares from *_pct_0..4 ---
            cols = pct_cols_by_asset[a]
            shares_df = df_mo[cols]

            # If upstream wiped pct to NaN for this muni+asset, fall back to gray
            if not shares_df.notna().any().any():
                ax.pie([1.0], labels=None, colors=["#e0e0e0"], startangle=90,
                    counterclock=True, wedgeprops=dict(width=0.5))
                ax.text(0, 0, format_center(a, float(center_val)), ha="center", va="center",
                        fontsize=11, weight="bold")
                continue

            shares_series = shares_df.sum(numeric_only=True)
            shares_series.index = [cat_idx_to_name[int(col.split("_")[-1])] for col in shares_series.index]
            shares_series = shares_series.reindex(category_order, fill_value=0.0)

            # --- IMPORTANT: avoid zero-sum pies ---
            total_sum = float(np.nansum(shares_series.values))
            if not np.isfinite(total_sum) or total_sum <= 0:
                # assets exist but no distribution recorded → 100% "неопасно"
                shares_series.loc[:] = 0.0
                shares_series["неопасно"] = 100.0
            elif total_sum < 100:
                # top-up remainder into "неопасно"
                shares_series["неопасно"] += (100 - total_sum)

            shares = [float(x) for x in shares_series.tolist()]

            # center = municipality’s Economy total (not damage)
            ax.text(0, 0, format_center(a, float(center_val)),
                    ha="center", va="center", fontsize=11, weight="bold")

            colors = [category_colors[k] for k in category_order]
            ax.pie(
                shares,
                labels=None,
                autopct=autopct_fmt,   # your smart % formatter
                colors=colors,
                startangle=90,
                counterclock=True,
                wedgeprops=dict(width=0.5),
                pctdistance=1.2,
                textprops=dict(fontsize=16),
            )

        # --- add y-axis label only once per row ---
        axes[r][0].set_ylabel(
            mo.replace(" ", "\n"),   # line break for long names
            fontsize=13, weight="bold", rotation=0, labelpad=40, va="center"
        )

    # layout & save
    plt.subplots_adjust(hspace=0.4, wspace=0.25)
    plt.tight_layout(rect=[0.04, 0.04, 0.98, 0.96])

    Path(base_dir).mkdir(parents=True, exist_ok=True)
    svg_path = f"{base_dir}{risk}_donuts_by_municipality_wide.svg"
    png_path = f"{base_dir}{risk}_donuts_by_municipality_wide.png"
    plt.savefig(svg_path, bbox_inches="tight")
    plt.savefig(png_path, bbox_inches="tight", dpi=180)
    plt.show()

    print(svg_path, png_path)

Heat


<ipython-input-19-f32c18669163>:181: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Heat_donuts_by_municipality_wide.svg C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Heat_donuts_by_municipality_wide.png
Cold
C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Cold_donuts_by_municipality_wide.svg C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Cold_donuts_by_municipality_wide.png
Precipitation
C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Precipitation_donuts_by_municipality_wide.svg C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Precipitation_donuts_by_municipality_wide.png
Wind
C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Wind_donuts_by_municipality_wide.svg C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Wind_donuts_by_municipality_wide.png
Freeze
C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Freeze_donuts_by_municipality_wide.svg C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Freeze_donuts_by_municipality_wide.png
Hail
C:\Users\sdfsg\Documents\work\Perm_RiskProfile2Hail_donuts_by_municipality_wide.svg C:\Users\sdfsg\Doc